# Data Preparation

In [1]:
from numpy import asarray
from numpy import save
from predictor.driver import Predictor
from predictor.src import utils

import numpy as np

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

training_data, names_list, scores_list = utils.load_training_data()
X, y = zip(*training_data)
X = list(X)
y = list(y)
for i in range(len(X)):
    X[i] = list(X[i])
    scores_list[i] = list(scores_list[i])


X, y, indices = np.array(X), np.array(y), np.array(range(len(y)))

non_zero_indices = []
for i in range(len(X[0])):
    if sum(X[:, i]) > 0:
        non_zero_indices.append(i)
X = X[:, non_zero_indices]

(
    X_train,
    X_test,
    y_train,
    y_test,
    indices_train,
    indices_test,
) = train_test_split(X, y, indices, test_size=0.3, random_state=5)

scores_filtered = [scores_list[i] for i in indices_test]
names_filtered = [names_list[i] for i in indices_test]


# scaler = preprocessing.StandardScaler().fit(X_train)
# in_max_scaler = preprocessing.MinMaxScaler()

# X_train = preprocessing.normalize(X_train, norm='l2')

# m = np.mean(X_train, axis=0) # array([16.25, 26.25])
# std = np.std(X_train, axis=0) # array([17.45530005, 22.18529919])

# X_train = 0.5 * (np.tanh(0.01 * ((X_train - m) / std)) + 1)

predictor = Predictor()
performance = []

In [ ]:
import matplotlib.pyplot as plt

plt.hist(X[:, -7], bins=range(1, 130, 1));

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV

clf = RandomForestClassifier(random_state=0)

tree_param = [
    {
        "n_estimators": [100, 200, 500],
        "max_features": ["auto", "sqrt"],
        "max_depth": list(range(8, 30, 6)),
        "min_samples_split": list(range(2, 20, 6)),
        "min_samples_leaf": list(range(2, 20, 6)),
        "bootstrap": [True, False],
    },
]

clf = GridSearchCV(clf, tree_param, cv=5, n_jobs=8).fit(X_train, y_train)


y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="RandomForestClassifier"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Feature Importance: ", clf.best_estimator_.feature_importances_)
performance.append(("RandomForestClassifier", clf.best_score_, top3, max(res)))

### Feature Importances

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

openqasm_qc_list = utils.get_openqasm_gates()
feature_names = [openqasm_qc_list[i] for i in range(0, len(openqasm_qc_list))]
feature_names.append("num_qubits")
feature_names.append("depth")
feature_names.append("program_communication")
feature_names.append("critical_depth")
feature_names.append("entanglement_ratio")
feature_names.append("parallelism")
feature_names.append("liveness")
feature_names = [feature_names[i] for i in non_zero_indices]

importances = clf.best_estimator_.feature_importances_
std = np.std(
    [tree.feature_importances_ for tree in clf.best_estimator_.estimators_], axis=0
)

idx = np.argsort(-importances)

plt.figure(figsize=(8, 8))
plt.bar(np.array(feature_names)[idx], np.array(importances)[idx])
plt.errorbar(
    np.array(feature_names)[idx],
    np.array(importances)[idx],
    np.array(std)[idx],
    fmt="o",
    color="lightgreen",
)
plt.xticks(rotation=90, fontsize=18)
plt.yticks(fontsize=18)
plt.ylabel("Relative feature importance", fontsize=18)
plt.tight_layout()
plt.savefig("feature_importances.pdf")
plt.show()

In [ ]:
len(non_zero_indices)

In [ ]:
predictor.plot_eval_all_detailed_compact_normed(
    names_filtered, scores_filtered, y_pred, y_test
)

# GradientBoostingClassifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()

param_grid = {
    "learning_rate": [0.01, 0.1, 1],
}

clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="GradientBoostingClassifier"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
performance.append(("GradientBoostingClassifier", clf.best_score_, top3, max(res)))

# Current Decision Tree Classifier

In [ ]:
# predictor.train_decision_tree_classifier(X, y, names_list, scores_list);

In [ ]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV

clf = tree.DecisionTreeClassifier(random_state=5)

tree_param = [
    {
        "criterion": ["entropy", "gini"],
        "max_depth": list(range(1, 15, 1)),
        "min_samples_split": list(range(2, 20, 4)),
        "min_samples_leaf": list(range(2, 20, 4)),
        "max_leaf_nodes": list(range(2, 200, 40)),
        "max_features": list(range(1, len(non_zero_indices), 10)),
    },
]
clf = GridSearchCV(clf, tree_param, cv=5, n_jobs=8).fit(X_train, y_train)
y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="DecisionTreeClassifier"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
print("Feature Importance: ", clf.best_estimator_.feature_importances_)
performance.append(("DecisionTreeClassifier", clf.best_score_, top3, max(res)))

# Nearest Neighbor

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()
param_grid = dict(n_neighbors=range(1, 10, 1))
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="KNeighborsClassifier"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print(("Top 3: ", top3))
performance.append(("KNeighborsClassifier", clf.best_score_, top3, max(res)))

# SVM

In [ ]:
from sklearn import svm

clf = svm.SVC()
param_grid = {"C": [0.1, 1, 10], "gamma": [1, 0.1, 0.01], "kernel": ["rbf", "sigmoid"]}
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(scores_filtered, y_pred, y_test, filename="SVM")

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
performance.append(("SVM", clf.best_score_, top3, max(res)))

# Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
param_grid = {"var_smoothing": np.logspace(0, -9, num=100)}
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="GaussianNB"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
performance.append(("GaussianNB", clf.best_score_, top3, max(res)))

# SGD

In [ ]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss="hinge", penalty="l2", max_iter=200)

param_grid = {
    "average": [True, False],
    "l1_ratio": np.linspace(0, 1, num=10),
    "alpha": np.power(10, np.arange(-2, 1, dtype=float)),
}
clf = GridSearchCV(clf, param_grid, cv=5, n_jobs=8).fit(X_train, y_train)

y_pred = np.array(list(clf.predict(X_test)))
res = predictor.plot_eval_histogram(
    scores_filtered, y_pred, y_test, filename="SGDClassifier"
)

print("Best Accuracy: ", clf.best_score_)
top3 = (res.count(1) + res.count(2) + res.count(3)) / len(res)
print("Top 3: ", top3)
performance.append(("SGDClassifier", clf.best_score_, top3, max(res)))

# Save Performance Results

In [ ]:
print(performance)

filename = "performances.csv"
with open(filename, "w") as f:
    f.write("Algorithm, Accuracy, Top3, Worst Rank\n")
    for sublist in performance:
        line = "{}, {}, {}, {} \n".format(
            sublist[0], sublist[1], sublist[2], sublist[3]
        )
        f.write(line)

# Debugging

In [2]:
from mqt.bench import get_one_benchmark
from qiskit.test.mock.backends import FakeMontreal, FakeWashington
from qiskit.providers.fake_provider import FakeWashington as was2

In [3]:
def get_rigetti_c_map(circles: int = 4):
    """Returns a coupling map of the circular layout scheme used by Rigetti.
    Keyword arguments:
    circles -- number of circles, each one comprises 8 qubits
    """
    if circles != 10:
        c_map_rigetti = []
        for j in range(circles):
            for i in range(0, 7):
                c_map_rigetti.append([i + j * 8, i + 1 + j * 8])

                if i == 6:
                    c_map_rigetti.append([0 + j * 8, 7 + j * 8])

            if j != 0:
                c_map_rigetti.append([j * 8 - 6, j * 8 + 5])
                c_map_rigetti.append([j * 8 - 7, j * 8 + 6])

        inverted = [[item[1], item[0]] for item in c_map_rigetti]
        c_map_rigetti = c_map_rigetti + inverted
    else:
        c_map_rigetti = []
        for j in range(5):
            for i in range(0, 7):
                c_map_rigetti.append([i + j * 8, i + 1 + j * 8])

                if i == 6:
                    c_map_rigetti.append([0 + j * 8, 7 + j * 8])

            if j != 0:
                c_map_rigetti.append([j * 8 - 6, j * 8 + 5])
                c_map_rigetti.append([j * 8 - 7, j * 8 + 6])

        for j in range(5):
            m = 8 * j + 5 * 8
            for i in range(0, 7):
                c_map_rigetti.append([i + m, i + 1 + m])

                if i == 6:
                    c_map_rigetti.append([0 + m, 7 + m])

            if j != 0:
                c_map_rigetti.append([m - 6, m + 5])
                c_map_rigetti.append([m - 7, m + 6])

        for n in range(5):
            c_map_rigetti.append([n * 8 + 3, n * 8 + 5 * 8])
            c_map_rigetti.append([n * 8 + 4, n * 8 + 7 + 5 * 8])

        inverted = [[item[1], item[0]] for item in c_map_rigetti]
        c_map_rigetti = c_map_rigetti + inverted

    return c_map_rigetti

In [4]:
qc = get_one_benchmark("ghz", 1, 79)
from qiskit import transpile

path = "test.qasm"
utils.init_all_config_files()

True

In [5]:
qc_transpiled = transpile(
    qc,
    basis_gates=["rz", "sx", "x", "cx"],
    coupling_map=FakeWashington().configuration().coupling_map,
    layout_method="sabre",
    routing_method="sabre",
    optimization_level=2,
)
qc_transpiled.qasm(filename=path)
utils.calc_eval_score_for_qc(path, "ibm_washington")

0 rz 39 gate
1
0.0002121618382649799 sx 39 gate
1.0
0 rz 39 gate
0.999787838161735
0.010949753454380706 cx 39 33 gate
0.999787838161735
0.009660625182169064 cx 20 33 gate
0.9888404078271756
0.009660625182169064 cx 33 20 gate
0.9792875912821741
0.009660625182169064 cx 20 33 gate
0.9698270609172479
0.011280772388416932 cx 21 20 gate
0.9604579251902018
0.011280772388416932 cx 20 21 gate
0.94962321794748
0.011280772388416932 cx 21 20 gate
0.9389107345710583
0.016896399851802107 cx 21 22 gate
0.9283190962813208
0.013082111568090526 cx 22 23 gate
0.912633845640488
0.013082111568090526 cx 23 22 gate
0.9006946678510036
0.013082111568090526 cx 22 23 gate
0.8889116797173925
0.013282912393089152 cx 23 24 gate
0.8772828379491507
0.010458657433666751 cx 24 34 gate
0.8656299668687116
0.010458657433666751 cx 34 24 gate
0.8565766395809156
0.010458657433666751 cx 24 34 gate
0.8476179979418574
0.027622457433062103 cx 38 39 gate
0.8387530516667732
0.027622457433062103 cx 39 38 gate
0.8155846312002568
0.0

0.010949753454380706 cx 39 33 gate
0.0
0.009660625182169064 cx 33 20 gate
0.0
0.011280772388416932 cx 21 20 gate
0.0
0.011280772388416932 cx 20 21 gate
0.0
0.011280772388416932 cx 21 20 gate
0.0
0.016896399851802107 cx 22 21 gate
0.0
0.016896399851802107 cx 21 22 gate
0.0
0.016896399851802107 cx 22 21 gate
0.0
0.013082111568090526 cx 23 22 gate
0.0
0.013082111568090526 cx 22 23 gate
0.0
0.013082111568090526 cx 23 22 gate
0.0
0.013282912393089152 cx 24 23 gate
0.0
0.013282912393089152 cx 23 24 gate
0.0
0.013282912393089152 cx 24 23 gate
0.0
0.008784874028099604 cx 24 25 gate
0.0
0.06943301362709067 cx 25 26 gate
0.0
0.06943301362709067 cx 26 25 gate
0.0
0.06943301362709067 cx 25 26 gate
0.0
0.010007523215808029 cx 26 27 gate
0.0
0.013700604202187272 cx 27 28 gate
0.0
0.013700604202187272 cx 28 27 gate
0.0
0.013700604202187272 cx 27 28 gate
0.0
0.006709616852062478 cx 29 28 gate
0.0
0.006709616852062478 cx 28 29 gate
0.0
0.006709616852062478 cx 29 28 gate
0.0
0.007609366756157171 cx 29 3

0.22899999999999998 measure 119 readout
0.0
0.008600000000000052 measure 79 readout
0.0
0.006299999999999972 measure 80 readout
0.0
0.008499999999999952 measure 85 readout
0.0
0.006000000000000005 measure 65 readout
0.0
0.034499999999999975 measure 21 readout
0.0
0.0041999999999999815 measure 20 readout
0.0
0.052000000000000046 measure 38 readout
0.0


0.0

In [6]:
qc_transpiled.count_ops()

OrderedDict([('cx', 847),
             ('measure', 79),
             ('rz', 2),
             ('sx', 1),
             ('barrier', 1)])

In [ ]:
qc_transpiled_rigetti = transpile(
    qc,
    basis_gates=["rx", "rz", "cz", "measure", "barrier"],
    coupling_map=get_rigetti_c_map(10),
)
qc_transpiled_rigetti.qasm(filename=path)
utils.calc_eval_score_for_qc(path, "rigetti_aspen_m1")

In [ ]:
qc_transpiled_rigetti.count_ops()

In [ ]:
qc = get_one_benchmark("ae", 1, 5)

In [12]:
qc.draw()

global phase: π
           ┌──────────┐                                                      »
eval_0: ───┤ U2(0,-π) ├─────■──────────────────────■─────────────────────────»
           ├──────────┤     │                      │                         »
eval_1: ───┤ U2(0,-π) ├─────┼──────────────────────┼──────────────────────■──»
           ├──────────┤     │                      │                      │  »
eval_2: ───┤ U2(0,-π) ├─────┼──────────────────────┼──────────────────────┼──»
           ├──────────┤     │                      │                      │  »
eval_3: ───┤ U2(0,-π) ├─────┼──────────────────────┼──────────────────────┼──»
        ┌──┴──────────┴──┐┌─┴─┐┌────────────────┐┌─┴─┐┌────────────────┐┌─┴─┐»
     q: ┤ U3(0.9273,0,0) ├┤ X ├┤ U(-0.9273,0,0) ├┤ X ├┤ U3(0.9273,0,0) ├┤ X ├»
        └────────────────┘└───┘└────────────────┘└───┘└────────────────┘└───┘»
meas: 5/═════════════════════════════════════════════════════════════════════»
                                                                             »
«                                                                             »
«eval_0: ─────────────────────────────────────────────────────────────────────»
«                                                                             »
«eval_1: ────────────────────■────────────────────────────────────────────────»
«                            │                                                »
«eval_2: ────────────────────┼──────────────────────■──────────────────────■──»
«                            │                      │                      │  »
«eval_3: ────────────────────┼──────────────────────┼──────────────────────┼──»
«        ┌────────────────┐┌─┴─┐┌────────────────┐┌─┴─┐┌────────────────┐┌─┴─┐»
«     q: ┤ U(-1.8546,0,0) ├┤ X ├┤ U3(1.8546,0,0) ├┤ X ├┤ U(-3.7092,0,0) ├┤ X ├»
«        └────────────────┘└───┘└────────────────┘└───┘└────────────────┘└───┘»
«meas: 5/═════════════════════════════════════════════════════════════════════»
«                                                                             »
«                                                                        »
«eval_0: ────────────────────────────────────────────────────────────────»
«                                                                        »
«eval_1: ────────────────────────────────────────────────────────────────»
«                                                                        »
«eval_2: ────────────────────────────────────────────────────────────────»
«                                                             ┌───┐      »
«eval_3: ─────────────────────■──────────────────────■────────┤ H ├──────»
«        ┌─────────────────┐┌─┴─┐┌────────────────┐┌─┴─┐┌─────┴───┴─────┐»
«     q: ┤ U3(2.574,-π,-π) ├┤ X ├┤ U(-7.4184,0,0) ├┤ X ├┤ U(7.4184,0,0) ├»
«        └─────────────────┘└───┘└────────────────┘└───┘└───────────────┘»
«meas: 5/════════════════════════════════════════════════════════════════»
«                                                                        »
«                                                                              »
«eval_0: ──────────────────────────■───────────────────■──────────────■────────»
«                                  │                   │        ┌───┐ │P(-π/2) »
«eval_1: ───────────■──────────────┼─────────■─────────┼────────┤ H ├─■────────»
«                   │        ┌───┐ │         │P(-π/2)  │P(-π/4) └───┘          »
«eval_2: ─■─────────┼────────┤ H ├─┼─────────■─────────■───────────────────────»
«         │P(-π/2)  │P(-π/4) └───┘ │P(-π/8)                                    »
«eval_3: ─■─────────■──────────────■───────────────────────────────────────────»
«                                                                              »
«     q: ──────────────────────────────────────────────────────────────────────»
«                                                                              »
«meas: 5/══════════════════════════════════════════

In [13]:
qc.count_ops()

OrderedDict([('cx', 8),
             ('cp', 6),
             ('u', 5),
             ('measure', 5),
             ('u2', 4),
             ('u3', 4),
             ('h', 4),
             ('barrier', 1)])

In [14]:
sum(qc.count_ops().values())

37

In [17]:
from qiskit.converters import circuit_to_dag

dag_circuit = circuit_to_dag(qc)
dag_circuit.count_ops_longest_path()

{'u2': 1,
 'cx': 8,
 'u': 4,
 'u3': 3,
 'h': 2,
 'cp': 5,
 'barrier': 1,
 'measure': 1}

In [21]:
sum(qc.count_ops())

25

In [23]:
a = utils.create_feature_dict(qc.qasm())

In [25]:
print(a.keys())

dict_keys(['tdg', 'z', 'cry', 'rxx', 'sxdg', 'crz', 'cu1', 'cu3', 'cx', 'u3', 'cswap', 'u', 'cu', 'u2', 'p', 'c3x', 'rzz', 'id', 'y', 's', 'sdg', 'ccx', 'rx', 'cz', 'rc3x', 'ry', 'rz', 'h', 'swap', 'cy', 'crx', 'sx', 'x', 'csx', 'c4x', 't', 'cp', 'u0', 'u1', 'c3sqrtx', 'rccx', 'ch', 'num_qubits', 'depth', '1_max_interactions', '2_max_interactions', '3_max_interactions', '4_max_interactions', '5_max_interactions'])


In [26]:
len(a)

49